In [10]:
import json

from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model_path="base_models/granite-3.2-2b-instruct"
device= "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_path
)

# model.save_pretrained("base_models/granite-3.2-2b-instruct")
# tokenizer.save_pretrained("base_models/granite-3.2-2b-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [120]:
conv = [{"role": "user", "content":"Translate to english: കണ്ണടയും തൊപ്പിയും ധരിച്ച ഒരാളുടെ ക്ലോസ് അപ്പ്"}]

input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", thinking=False, return_dict=True, add_generation_prompt=True).to(device)
set_seed(42)
print(tokenizer.decode(input_ids['input_ids'][0]))

import sys; sys.exit(0)

output = model.generate(
    **input_ids,
    max_new_tokens=8192,
)

prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
print(prediction)

<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.
Today's Date: March 04, 2025.
You are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>Translate to english: കണ്ണടയും തൊപ്പിയും ധരിച്ച ഒരാളുടെ ക്ലോസ് അപ്പ്<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>


SystemExit: 0

In [14]:
from datasets import Dataset

def preprocess_data(sources, destinations):
    dataset = [[{"role": "user", "content": f"Translate to english: {src}"} , {"role": "assistant", "content": dest}] for src, dest in zip(sources, destinations)]
    dataset = Dataset.from_dict({"formatted_chat": dataset})
    return dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x, tokenize=False, add_generation_prompt=False)})

def get_translation_loss(examples):
    input_ids = tokenizer.apply_chat_template(examples, return_tensors="pt", thinking=False, return_dict=True, add_generation_prompt=True).to(device)
    print(input_ids)
    

with open("data/coco.ml.txt", "r") as f:
    src_data = f.readlines()

with open("data/coco.en.txt", "r") as f:
    dest_data = f.readlines()

data = preprocess_data(src_data[:10], dest_data[:10])
data

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

UndefinedError: datasets.formatting.formatting.LazyRow object has no element 0

In [86]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-2b-instruct")

In [85]:
count, n = 0, 10000
for i in range(n):
    sample = src_data[i]
    count += tokenizer.decode(tokenizer.encode(sample)) == sample

count / n

0.9989

In [54]:
# Dataset preparation

# {
#     "messages": [
#         {"role": "system", "content": "Translate the Malayalam text to English."},
#         {"role": "user", "content": "മലയാളം ടെക്സ്റ്റ്"},
#         {"role": "assistant", "content": "Malayalam text"}
#     ]
# }

from ipywidgets import IntProgress

n = len(src_data)
dataset = []
for mal, eng in zip(src_data[:n], dest_data[:n]):
    sample = {
        "messages": [
            {"role": "system", "content": "Translate the Malayalam text to English."},
            {"role": "user", "content": mal.strip()},
            {"role": "assistant", "content": eng.strip()}
        ]
    }
    dataset.append({
        "input": mal.strip(),
        "output": eng.strip()
    })

import json

with open("data/trans.jsonl", "w") as f:
    for sample in dataset:
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-2b-instruct")

sample = {
    "messages": [
        {"role": "system", "content": "Translate Malayalam to English."},
        {"role": "user", "content": "മലയാളം ടെക്സ്റ്റ്"},
        {"role": "assistant", "content": "Malayalam text"}
    ]
    
}

print(tokenizer.apply_chat_template(sample["messages"], tokenize=False))

<|start_of_role|>system<|end_of_role|>Translate Malayalam to English.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>മലയാളം ടെക്സ്റ്റ്<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>Malayalam text<|end_of_text|>



In [45]:
element = {'messages': [{'role': 'system', 'content': 'Translate the Malayalam text to English.'}, {'role': 'user', 'content': 'ഒരു വലിയ...യിൻ ഒരു ട്രെയിൻ സ്റ്റേഷനിൽ ഇരിക്കുന്നു.'}, {'role': 'assistant', 'content': 'A large freight train sits in a train station.'}]}

delimiter_token = 17594
tokens = tokenizer.apply_chat_template(element['messages'], tokenize=True)
input_ids = tokens[:tokens.index(delimiter_token) + 2]
labels = tokens[tokens.index(delimiter_token) + 2:]
tokenizer.decode(input_ids), tokenizer.decode(labels)

('<|start_of_role|>system<|end_of_role|>Translate the Malayalam text to English.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>ഒരു വലിയ...യിൻ ഒരു ട്രെയിൻ സ്റ്റേഷനിൽ ഇരിക്കുന്നു.<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>',
 'A large freight train sits in a train station.<|end_of_text|>\n')

In [42]:
tokenizer.apply_chat_template(element['messages'], tokenize=False)

'<|start_of_role|>system<|end_of_role|>Translate the Malayalam text to English.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>ഒരു വലിയ...യിൻ ഒരു ട്രെയിൻ സ്റ്റേഷനിൽ ഇരിക്കുന്നു.<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>A large freight train sits in a train station.<|end_of_text|>\n'

In [27]:
tokenizer.chat_template

'{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\'] %}\n    {%- set loop_messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "Knowledge Cutoff Date: April 2024.\nToday\'s Date: " + strftime_now(\'%B %d, %Y\') + ".\nYou are Granite, developed by IBM." %}\n    {%- if tools and documents %}\n        {%- set system_message = system_message + " You are a helpful AI assistant with access to the following tools. When a tool is required to answer the user\'s query, respond with <|tool_call|> followed by a JSON list of tools used. If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.\n\nWrite the response to the user\'s input by strictly aligning with the facts in the provided documents. If the information needed to answer the question is not available in the documents, inform the user that the question cannot be answered based on the available data." %}

In [48]:
model.modules

<bound method Module.modules of GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49155, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
      )
    

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for peft==0.13.2 from https://files.pythonhosted.org/packages/78/9d/5f95bfb298c8d3b4e3a107701f9a4e7774a0d4d1f8eb0c9d5420b80f7c9d/peft-0.13.2-py3-none-any.whl.metadata
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for accelerate==1.0.1 from https://files.pythonhosted.org/packages/2c/92/48aec3736ca778ffe5fa68e19e3c18917cba4de43fa46fe6176cccafe267/accelerate-1.0.1-py3-none-any.whl.metadata
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
ERROR: Cannot install -r requirements.txt (line 1), -r requirements.txt (line 86), torch==2.4.1 and torchaudio==2.6.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.4.1
    accelerate 1.0.1 depends on torch>=1.10.0
    peft 0.13.2 depend